# LANS - Summarization

## Load the dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0')

dataset['train'] = dataset['train'].shuffle(seed=42).select(range(50000))
dataset['validation'] = dataset['validation'].shuffle(seed=42).select(range(5000))
dataset['test'] = dataset['test'].shuffle(seed=42).select(range(5000))

Found cached dataset cnn_dailymail (C:/Users/jayar/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00,  6.31it/s]
Loading cached shuffled indices for dataset at C:/Users/jayar/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de\cache-380ef232430d96da.arrow
Loading cached shuffled indices for dataset at C:/Users/jayar/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de\cache-fe8ce4e4283768ec.arrow
Loading cached shuffled indices for dataset at C:/Users/jayar/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de\cache-914896ee6a47e4aa.arrow


## Data preprocessing

In [2]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
MODEL_NAME = 't5-small'
PREFIX  = 'summarize: '

ENCODING_LEN = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

c:\Softwares\Anaconda\envs\torch\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned if len(sent) > 0 and sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples['article']]
  inputs = [f'{PREFIX} {text}' for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=ENCODING_LEN, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['highlights'], max_length=ENCODING_LEN, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [5]:
dataset_cleaned = dataset.filter(lambda example: (len(example['article']) >= 500) and (example['highlights'] is not None and len(example['highlights']) >= 20))
tokenized_datasets = dataset_cleaned.map(preprocess_data, batched=True)
tokenized_datasets

  0%|          | 0/50 [00:00<?, ?ba/s]c:\Softwares\Anaconda\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██████████| 5/5 [00:05<00:00,  1.14s/ba]


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 49954
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4996
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4996
    })
})

## Fine-tune T5-Small

In [6]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [7]:
EPOCHS = 10
LR = 2e-5
BATCH_SIZE = 16
WEIGHT_DECAY = 0.01
MODEL_DIR = f".models/summarization/{MODEL_NAME}/ver2-large"


args = Seq2SeqTrainingArguments(
    MODEL_DIR,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=3,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
)

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

C:\Users\jayar\AppData\Local\Temp\ipykernel_23860\1756815915.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [10]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at C:\Users\jayar/.cache\huggingface\hub\models--t5-small\snapshots\d78aea13fa7ecd06c29e3e46195d6341255065d5\config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

In [11]:
trainer.train()

loading configuration file config.json from cache at C:\Users\jayar/.cache\huggingface\hub\models--t5-small\snapshots\d78aea13fa7ecd06c29e3e46195d6341255065d5\config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

{'eval_loss': 2.6503148078918457, 'eval_rouge1': 23.1116, 'eval_rouge2': 10.4515, 'eval_rougeL': 18.8978, 'eval_rougeLsum': 21.6546, 'eval_gen_len': 19.0, 'eval_runtime': 77.4651, 'eval_samples_per_second': 64.494, 'eval_steps_per_second': 4.041, 'epoch': 0.03}


  1%|          | 200/31230 [01:42<1:02:19,  8.30it/s]  The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4996
  Batch size = 16
                                                     
  1%|          | 200/31230 [02:56<1:02:19,  8.30it/s]Saving model checkpoint to .models/summarization/t5-small/ver2-large\checkpoint-200
Configuration saved in .models/summarization/t5-small/ver2-large\checkpoint-200\config.json


{'eval_loss': 2.53800106048584, 'eval_rouge1': 23.1092, 'eval_rouge2': 10.4218, 'eval_rougeL': 18.9128, 'eval_rougeLsum': 21.6933, 'eval_gen_len': 18.9998, 'eval_runtime': 73.886, 'eval_samples_per_second': 67.618, 'eval_steps_per_second': 4.236, 'epoch': 0.06}


Model weights saved in .models/summarization/t5-small/ver2-large\checkpoint-200\pytorch_model.bin
tokenizer config file saved in .models/summarization/t5-small/ver2-large\checkpoint-200\tokenizer_config.json
Special tokens file saved in .models/summarization/t5-small/ver2-large\checkpoint-200\special_tokens_map.json
Copy vocab file to .models/summarization/t5-small/ver2-large\checkpoint-200\spiece.model
  1%|          | 300/31230 [03:10<1:04:40,  7.97it/s]  The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4996
  Batch size = 16
                                                     
  1%|          | 300/31230 [04:31<1:04:40,  7.97it/s]

...

100%|█████████▉| 31199/31230 [7:35:35<00:03,  8.84it/s]The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4996
  Batch size = 16
                                                       
100%|█████████▉| 31200/31230 [7:36:53<00:03,  8.84it/s]Saving model checkpoint to .models/summarization/t5-small/ver2-large\checkpoint-31200
Configuration saved in .models/summarization/t5-small/ver2-large\checkpoint-31200\config.json


{'eval_loss': 2.4050023555755615, 'eval_rouge1': 23.3876, 'eval_rouge2': 10.7827, 'eval_rougeL': 19.1937, 'eval_rougeLsum': 21.9878, 'eval_gen_len': 18.9996, 'eval_runtime': 77.6702, 'eval_samples_per_second': 64.323, 'eval_steps_per_second': 4.03, 'epoch': 9.99}


Model weights saved in .models/summarization/t5-small/ver2-large\checkpoint-31200\pytorch_model.bin
tokenizer config file saved in .models/summarization/t5-small/ver2-large\checkpoint-31200\tokenizer_config.json
Special tokens file saved in .models/summarization/t5-small/ver2-large\checkpoint-31200\special_tokens_map.json
Copy vocab file to .models/summarization/t5-small/ver2-large\checkpoint-31200\spiece.model
Deleting older checkpoint [.models\summarization\t5-small\ver2-large\checkpoint-30800] due to args.save_total_limit
100%|█████████▉| 31229/31230 [7:36:59<00:00,  8.58it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from .models/summarization/t5-small/ver2-large\checkpoint-6400 (score: 23.5011).
100%|██████████| 31230/31230 [7:36:59<00:00,  1.14it/s]

{'train_runtime': 27419.8587, 'train_samples_per_second': 18.218, 'train_steps_per_second': 1.139, 'train_loss': 2.7741234956778587, 'epoch': 10.0}


TrainOutput(global_step=31230, training_loss=2.7741234956778587, metrics={'train_runtime': 27419.8587, 'train_samples_per_second': 18.218, 'train_steps_per_second': 1.139, 'train_loss': 2.7741234956778587, 'epoch': 10.0})

In [12]:
trainer.save_model()

Saving model checkpoint to .models/summarization/t5-small/ver2-large
Configuration saved in .models/summarization/t5-small/ver2-large\config.json
Model weights saved in .models/summarization/t5-small/ver2-large\pytorch_model.bin
tokenizer config file saved in .models/summarization/t5-small/ver2-large\tokenizer_config.json
Special tokens file saved in .models/summarization/t5-small/ver2-large\special_tokens_map.json
Copy vocab file to .models/summarization/t5-small/ver2-large\spiece.model
